In [23]:
import tweepy
import configparser
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle
import nltk
import string
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
import json

In [2]:
# read data 
train_data_file = open('project-data/train.data.txt', 'r')
train_lines = train_data_file.readlines()
train_events =[]
# Strips the newline character
for line in train_lines:
    train_events.append(list(map(int,line.strip('\n').split(','))))
    
train_label_file = open('project-data/train.label.txt', 'r')
train_labels = train_label_file.readlines()
train_labels = [label.strip('\n') for label in train_labels]


dev_data_file = open('project-data/dev.data.txt', 'r')
dev_lines = dev_data_file.readlines()
dev_events =[]
# Strips the newline character
for line in dev_lines:
    dev_events.append(list(map(int,line.strip('\n').split(','))))
    
dev_label_file = open('project-data/dev.label.txt', 'r')
dev_labels = dev_label_file.readlines()
dev_labels = [label.strip('\n') for label in dev_labels]

In [3]:
# open train text file
f = open(f'./tweet_text.pckl','rb')
train_data = pickle.load(f)
f.close()


# open dev text file
f = open(f'./dev_tweet_text.pckl','rb')
dev_data = pickle.load(f)
f.close()

In [4]:
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mention
    text = re.sub(r'#','',text) # remove the hashtag symbol
    text = re.sub(r'https?:\/\/\S+', '',text) #remove hyperlink
    text = re.sub(r'\n','',text) # remove \n 
    return text

for i in range(len(train_data)):
    for j in range(len(train_data[i])):
        train_data[i][j] = clean_text(train_data[i][j])
        
for i in range(len(dev_data)):
    for j in range(len(dev_data[i])):
        dev_data[i][j] = clean_text(dev_data[i][j])

In [5]:
# merge source tweeet and reply tweet together for train data
train_merge_events=[]
for event in train_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    train_merge_events.append(merge)
    
    
# merge source tweeet and reply tweet together for dev data
dev_merge_events=[]
for event in dev_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    dev_merge_events.append(merge)

In [6]:
def tokenize_tweet(tweet):
    """Get all of the tokens in a set of tweets"""
    twt = nltk.tokenize.TweetTokenizer()
    # combine stop words and punctuation
    stopwords = nltk.corpus.stopwords.words('english')
    stop = stopwords + list(string.punctuation)
    # filter out stop words and punctuation and send to lower case
    tokens = [token.lower() for token in twt.tokenize(tweet)
              if token.lower() not in stop]
    tokens = [word for word in tokens if re.search('[a-zA-Z]',word) is not None] # filter out word not contain alphabet
    return(tokens)

In [7]:
def tokenize_tweetv2(tweet):
    """Get all of the tokens in a set of tweets"""
    twt = nltk.tokenize.TweetTokenizer()
    # combine stop words and punctuation
    stopwords = nltk.corpus.stopwords.words('english')
    stop = stopwords + list(string.punctuation)
    # create the stemmer
    stemmer = nltk.stem.porter.PorterStemmer()
    # filter out stop words and punctuation and send to lower case
    tokens = [ stemmer.stem(token) for token in twt.tokenize(tweet)
              if token.lower() not in stop]
    return(tokens)

### Normal bag of word

In [8]:
# Create bag of word 
def bow(data,labels):
    x = []
    y = []
    for i in range(len(data)):
        tokens = tokenize_tweet(data[i])
        
        vocab = collections.defaultdict(int)
        for word in tokens:
            vocab[word] += 1 
        x.append(vocab)
        y.append(labels[i])
    return x,y
    

In [9]:
x_train,y_train = bow(train_merge_events,train_labels)
x_dev,y_dev = bow(dev_merge_events,dev_labels)

In [10]:
vectorizer = DictVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_dev = vectorizer.transform(x_dev)

In [11]:
# k fold to find the optimize hyperparameter
alphas = [0.001,0.005,0.01,0.1,0.3,0.5,1]
max_nb = 0
for alpha in alphas:
    nb = MultinomialNB(alpha=alpha)
    nb_predict = nb.fit(x_train, y_train).predict(x_dev)    
    nb_accuracy = accuracy_score(y_dev,nb_predict)
    print('With alpha = {alpha} the accuracy of Naive Bayes is {acc:.5f}'.format(alpha=alpha, acc = nb_accuracy))
    if nb_accuracy > max_nb:
        max_nb = nb_accuracy
        max_alpha = alpha
print("The best setting for Naive Bayes is alpha = {alpha} with accuracy = {acc:.5f}".format(alpha=max_alpha,acc=max_nb))


With alpha = 0.001 the accuracy of Naive Bayes is 0.90032
With alpha = 0.005 the accuracy of Naive Bayes is 0.89715
With alpha = 0.01 the accuracy of Naive Bayes is 0.88924
With alpha = 0.1 the accuracy of Naive Bayes is 0.88608
With alpha = 0.3 the accuracy of Naive Bayes is 0.87816
With alpha = 0.5 the accuracy of Naive Bayes is 0.88133
With alpha = 1 the accuracy of Naive Bayes is 0.88608
The best setting for Naive Bayes is alpha = 0.001 with accuracy = 0.90032


In [12]:
solvers = ['newton-cg', 'lbfgs', 'liblinear','sag','saga']
c_values = [ 100,10,1.0, 0.1, 0.01,0.001]
max_lr = 0
for solver in solvers:
    print('Using this solver ',solver )
    for c_value in c_values:
        lr = LogisticRegression(C=c_value, penalty='l2', solver=solver,max_iter=1000)
        lr_predict = lr.fit(x_train, y_train).predict(x_dev)    
        lr_accuracy = accuracy_score(y_dev,lr_predict)
        print('With C = {c} and solver  = {sol} the acciracy of Logistic Regression is {acc}'.format(c=c_value,sol=solver,acc= lr_accuracy))
        if lr_accuracy > max_lr:
            max_lr = lr_accuracy
            max_c_value = c_value
            max_solver = solver
print("The best setting for Logistic Regression is c = {c} and solver = {sol} with accuracy = {acc:.5f}".format(c=max_c_value,sol=max_solver,acc=max_lr))

Using this solver  newton-cg
With C = 100 and solver  = newton-cg the acciracy of Logistic Regression is 0.9113924050632911
With C = 10 and solver  = newton-cg the acciracy of Logistic Regression is 0.9145569620253164
With C = 1.0 and solver  = newton-cg the acciracy of Logistic Regression is 0.9129746835443038
With C = 0.1 and solver  = newton-cg the acciracy of Logistic Regression is 0.8876582278481012
With C = 0.01 and solver  = newton-cg the acciracy of Logistic Regression is 0.8433544303797469
With C = 0.001 and solver  = newton-cg the acciracy of Logistic Regression is 0.7958860759493671
Using this solver  lbfgs
With C = 100 and solver  = lbfgs the acciracy of Logistic Regression is 0.9113924050632911
With C = 10 and solver  = lbfgs the acciracy of Logistic Regression is 0.9145569620253164
With C = 1.0 and solver  = lbfgs the acciracy of Logistic Regression is 0.9129746835443038
With C = 0.1 and solver  = lbfgs the acciracy of Logistic Regression is 0.8876582278481012
With C = 0.

D:\Anaconda\envs\CV\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


With C = 100 and solver  = sag the acciracy of Logistic Regression is 0.9272151898734177


D:\Anaconda\envs\CV\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


With C = 10 and solver  = sag the acciracy of Logistic Regression is 0.9272151898734177


...

With C = 10 and solver  = saga the acciracy of Logistic Regression is 0.9272151898734177


D:\Anaconda\envs\CV\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


With C = 1.0 and solver  = saga the acciracy of Logistic Regression is 0.9272151898734177


D:\Anaconda\envs\CV\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


With C = 0.1 and solver  = saga the acciracy of Logistic Regression is 0.9240506329113924
With C = 0.01 and solver  = saga the acciracy of Logistic Regression is 0.9145569620253164
With C = 0.001 and solver  = saga the acciracy of Logistic Regression is 0.8718354430379747
The best setting for Logistic Regression is c = 100 and solver = sag with accuracy = 0.92722


In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

clfs = [KNeighborsClassifier(),DecisionTreeClassifier(),RandomForestClassifier(),
        MultinomialNB(),LinearSVC(),LogisticRegression()]


In [14]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score, classification_report

def do_multiple_10foldcrossvalidation(clfs,data,classifications):
    for clf in clfs:
        predictions = model_selection.cross_val_predict(clf, data,classifications, cv=10)
        print (clf)
        print ("accuracy")
        print (accuracy_score(classifications,predictions))
        print (classification_report(classifications,predictions))
        
do_multiple_10foldcrossvalidation(clfs,x_train,y_train)


KNeighborsClassifier()
accuracy
0.8158311345646438
              precision    recall  f1-score   support

   nonrumour       0.81      0.99      0.89      1475
      rumour       0.84      0.21      0.34       420

    accuracy                           0.82      1895
   macro avg       0.83      0.60      0.61      1895
weighted avg       0.82      0.82      0.77      1895

DecisionTreeClassifier()
accuracy
0.8321899736147758
              precision    recall  f1-score   support

   nonrumour       0.88      0.91      0.89      1475
      rumour       0.64      0.56      0.60       420

    accuracy                           0.83      1895
   macro avg       0.76      0.73      0.75      1895
weighted avg       0.83      0.83      0.83      1895

RandomForestClassifier()
accuracy
0.8390501319261213
              precision    recall  f1-score   support

   nonrumour       0.84      0.98      0.90      1475
      rumour       0.82      0.35      0.49       420

    accuracy             

D:\Anaconda\envs\CV\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda\envs\CV\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda\envs\CV\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda\envs\CV\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda\envs\CV\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()
accuracy
0.8722955145118734
              precision    recall  f1-score   support

   nonrumour       0.90      0.93      0.92      1475
      rumour       0.74      0.65      0.69       420

    accuracy                           0.87      1895
   macro avg       0.82      0.79      0.81      1895
weighted avg       0.87      0.87      0.87      1895

LogisticRegression()
accuracy
0.8949868073878628
              precision    recall  f1-score   support

   nonrumour       0.90      0.97      0.93      1475
      rumour       0.86      0.63      0.73       420

    accuracy                           0.89      1895
   macro avg       0.88      0.80      0.83      1895
weighted avg       0.89      0.89      0.89      1895



### Using td-idf

In [72]:
# need to write manually for better tokenize
td = TfidfVectorizer(stop_words='english')
x_train = td.fit_transform(train_merge_events)
x_dev = td.transform(dev_merge_events)

#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer(stop_words='english')
#x_train  = vectorizer.fit_transform(train_merge_events)
#x_dev = vectorizer.transform(dev_merge_events)

In [16]:
# k fold to find the optimize hyperparameter
alphas = [0.001,0.005,0.01,0.1,0.3,0.5,1]
max_nb = 0
for alpha in alphas:
    nb = MultinomialNB(alpha=alpha)
    nb_predict = nb.fit(x_train, y_train).predict(x_dev)    
    nb_accuracy = accuracy_score(y_dev,nb_predict)
    print('With alpha = {alpha} the accuracy of Naive Bayes is {acc:.5f}'.format(alpha=alpha, acc = nb_accuracy))
    if nb_accuracy > max_nb:
        max_nb = nb_accuracy
        max_alpha = alpha
print("The best setting for Naive Bayes is alpha = {alpha} with accuracy = {acc:.5f}".format(alpha=max_alpha,acc=max_nb))

With alpha = 0.001 the accuracy of Naive Bayes is 0.91456
With alpha = 0.005 the accuracy of Naive Bayes is 0.91772
With alpha = 0.01 the accuracy of Naive Bayes is 0.90981
With alpha = 0.1 the accuracy of Naive Bayes is 0.91772
With alpha = 0.3 the accuracy of Naive Bayes is 0.88608
With alpha = 0.5 the accuracy of Naive Bayes is 0.85285
With alpha = 1 the accuracy of Naive Bayes is 0.79905
The best setting for Naive Bayes is alpha = 0.005 with accuracy = 0.91772


In [73]:
def do_multiple_10foldcrossvalidation(clfs,data,classifications):
    for clf in clfs:
        predictions = model_selection.cross_val_predict(clf, data,classifications, cv=10)
        print (clf)
        print ("accuracy")
        print (accuracy_score(classifications,predictions))
        print (classification_report(classifications,predictions))
        
do_multiple_10foldcrossvalidation(clfs,x_train,y_train)

KNeighborsClassifier()
accuracy
0.7915567282321899
              precision    recall  f1-score   support

   nonrumour       0.79      1.00      0.88      1475
      rumour       0.86      0.07      0.13       420

    accuracy                           0.79      1895
   macro avg       0.82      0.53      0.51      1895
weighted avg       0.81      0.79      0.72      1895

DecisionTreeClassifier()
accuracy
0.8316622691292876
              precision    recall  f1-score   support

   nonrumour       0.88      0.91      0.89      1475
      rumour       0.63      0.57      0.60       420

    accuracy                           0.83      1895
   macro avg       0.76      0.74      0.75      1895
weighted avg       0.83      0.83      0.83      1895

RandomForestClassifier()
accuracy
0.8411609498680739
              precision    recall  f1-score   support

   nonrumour       0.84      0.99      0.91      1475
      rumour       0.90      0.32      0.47       420

    accuracy             

### Test

In [20]:
# read data 
test_data_file = open('project-data/test.data.txt', 'r')
test_lines = test_data_file.readlines()
test_events =[]
# Strips the newline character
for line in test_lines:
    test_events.append(list(map(int,line.strip('\n').split(','))))

In [30]:
test_data=[]
for event in test_events:
    tweet_list=[]
    for tweet in event: 
        f = open('project-data/tweet-objects/'+str(tweet)+'.json')
        data = json.load(f)
        tweet_list.append(data['text'])
    test_data.append(tweet_list)

In [33]:
for i in range(len(test_data)):
    for j in range(len(test_data[i])):
        test_data[i][j] = clean_text(test_data[i][j])

In [36]:
test_merge_events=[]
for event in test_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    test_merge_events.append(merge)

In [74]:
td = TfidfVectorizer(stop_words='english')
x_train = td.fit_transform(train_merge_events)
x_test = td.transform(test_merge_events)

In [40]:
nb = MultinomialNB(alpha= 0.005)
nb_predict = nb.fit(x_train, y_train).predict(x_test)    

In [68]:
predict=[]
for result in nb_predict:
    if result == "nonrumour":
        predict.append(0)
    else:
        predict.append(1)
df = pd.DataFrame({"Id": range(len(predict)),"Predicted": predict}) 
df.to_csv('nb_test.csv',index=False)

In [69]:
lr = LogisticRegression()
lr_predict = lr.fit(x_train, y_train).predict(x_test)    

In [70]:
predict=[]
for result in lr_predict:
    if result == "nonrumour":
        predict.append(0)
    else:
        predict.append(1)
df = pd.DataFrame({"Id": range(len(predict)),"Predicted": predict}) 
df.to_csv('lr_test.csv',index=False)

In [75]:
SVC = LinearSVC()
SVC_predict = SVC.fit(x_train, y_train).predict(x_test)    

In [78]:
# this is the highest score atm
predict=[]
for result in SVC_predict:
    if result == "nonrumour":
        predict.append(0)
    else:
        predict.append(1)
df = pd.DataFrame({"Id": range(len(predict)),"Predicted": predict}) 
df.to_csv('svc_test.csv',index=False)